# Profiling on the `icemodel` as an example



## A note on environments

Julia has the notion of "stacked environments", allowing for the seperation of development tools
being used from the REPL and actual project dependencies.

My `1.8` environment looks like:
```
Status `~/.julia/environments/v1.8/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.2
  [f68482b8] Cthulhu v2.4.5
  [e4faabce] PProf v2.0.1
  [295af30f] Revise v3.3.1
  [9abbd945] Profile
```

```julia
@time md = solve(md, "Transient")

# First run
29.688897 seconds (429.20 M allocations: 23.641 GiB, 11.78% gc time, 2.07% compilation time)

# Second run
28.472670 seconds (428.59 M allocations: 23.737 GiB, 9.37% gc time)
```

### Profiling steps

1. Have a good test-case that where you can tune the time spent
2. Make sure that the code has been compiled first, note the `compilation time` included in the first run.
3. `using Profile; @profile` for a first intuition where time is spent
4. Export / Visualizing

### Export

#### Profile.jl

```julia
data, lidict = Profile.retrieve()
# then write to disc using your favourite tool...
```

or

```julia
import ProfileView
ProfileView.view() # then save button
```

and to load the data later use the "open button"

#### PProf

```julia
using PProf
pprof(; out="initial-profile.pb.gz", web=false)
```

Loading the data later:

```julia
using PProf
PProf.refresh(file="initial-profile.pb.gz")
```

A note on ProfileView colours:

> It is also worth noting that red is (by default) a special color: it is reserved for function calls that have to be resolved at run-time. Because run-time dispatch (aka, dynamic dispatch, run-time method lookup, or a virtual call) often has a significant impact on performance,

### Task

Open the profile up in both `PProf` and `ProfileView` and inspect it.
Note the difference that `PProf` by default includes `C` frames whereas `ProfileView` hides them.

This is a preference, both can show `C` frames and both can hide them.

On flamegraphs: https://www.brendangregg.com/flamegraphs.html

## Coming in 1.8

1. Thread information (e.g. on which Julia worker thread is code running)
2. Allocation information



### Allocation profiler

Requires:
- Julia 1.8-dev
- PProf#nhd-profile-allocs

```
Profile.Allocs.@profile md = solve(md, "Transient")
PProf.Allocs.to_pprof(Profile.Allocs.fetch(); web = false, out="initial-alloc-profile.pb.gz")
```

We can visualize it without being on that branch

```
    ┌ Warning: The allocation profiler is not fully implemented, and missed approximately 37% (estimated 15865 / 42859) of sampled allocs in the last run. For more info see https://github.com/JuliaLang/julia/issues/43688
    └ @ Profile.Allocs ~/builds/julia/usr/share/julia/stdlib/v1.8/Profile/src/Allocs.jl:128
```


Additionally the Alloc profile is sub-sampling allocations to keep overheads managable:
```
Profile.Allocs.@profile sample_rate=1.0 md = solve(md, "Transient")
```

Is real slow